## Re-identification/De-identification 


Read a configuration file containing integers that correspond to column numbers in the
dataset. Each integer represents a quasi-identifier. 


In [1]:
def read_config(config_file):
    with open(config_file) as file:
        identifiers = [int(q) for q in file.read().split()]
    file.close()
    return sorted(identifiers)

### Import data

In [2]:
import pandas as pd

In [3]:
NA_FILL_VALUE = 0

In [4]:
df_raw = pd.read_csv("mid_sample_set.csv", dtype='unicode')
df_raw.index = df_raw.user_id
df_raw = df_raw.drop('user_id', axis = 1)

In [5]:
# Remove NA columns
original_columns = set(df_raw.columns.values)
df = df_raw.dropna(axis = 1, how = 'all').fillna(NA_FILL_VALUE)
new_columns = set(df.columns.values)
print("Removed columns", original_columns - new_columns)

Removed columns {'roles_isLibrary', 'forumRoles_isCommunityTA', 'roles_isCCX'}


In [6]:
df.shape

(199999, 87)

### Direct Identifiers
- Can uniquely identify an individual and should be removed
- This includes ip


In [7]:
df = df.drop('ip', axis = 1)

### Quasi-Identifiers
- Can uniquely identify an individual when linked to other datasets. 
- These include: 'course_id', 'user_id', 'countryLabel', 'continent', 'city', 'region', 'subdivision', 'postalCode', 'LoE', 'YoB', 'gender', 'nforum_posts', 'nforum_votes', 'nforum_endorsed', 'nforum_threads', 'nforum_comments', 'nforum_pinned', and 'nforum_events', and are listed as their corresponding column index in the configuration file.
- Create a version of the dataset that only contains the quasi-identifiers.

In [8]:
quasi_identifiers = read_config("config_file.txt")
df_quasi = df.iloc[:,quasi_identifiers]
labels = list(df_quasi.columns.values)
# Remove user_id because it's not a quasi-identifier, just a key
quasi_identifier_labels = list(set(labels) - set(["user_id"]))

In [9]:
def getAnonCompletionRates(df, quasi_identifier_labels, k):
    if 'course_id' not in quasi_identifier_labels:
        quasi_identifier_labels.append('course_id')
    
    k_anonymous = df.groupby(quasi_identifier_labels)\
                  .size().reset_index(name = 'ct').set_index(quasi_identifier_labels)
    k_anonymous = k_anonymous[k_anonymous.ct >= k]
    
    completedCount = df.groupby(quasi_identifier_labels + ['completed'])\
                       .size()\
                       .reset_index(name = "completedCount")\
                       .set_index(quasi_identifier_labels)
    
    k_anonymous = k_anonymous.reset_index()\
                  .merge(completedCount[completedCount.completed == 'True'].reset_index(), how = "left")
        
    print("Size of %d-anonymous dataset: %d" % (k, k_anonymous.ct.sum()))
    
    k_AnonTotalCounts = k_anonymous.groupby('course_id')["ct"].sum()\
                                   .reset_index(name = "numStudents")\
                                   .set_index('course_id')
    
    completionRates = ((k_anonymous.set_index('course_id').completedCount.fillna(0)
                        / k_AnonTotalCounts.numStudents).fillna(0) * 100)\
                        .reset_index(name = "completionRate")\
                        .sort_values(by = "completionRate", ascending = False)
    
    if len(k_anonymous) == 0: 
        completionRates.completionRate = 0
    
    return completionRates 

When k = 3

In [10]:
completionRate3Anon = getAnonCompletionRates(df, quasi_identifier_labels, 3)

Size of 3-anonymous dataset: 16303


In [11]:
completionRate3Anon.head()

,course_id,completionRate
1764,HarvardX/MUS24.1x/3T2017,33.333333
1673,HarvardX/HLS4X/3T2017,11.111111
2653,HarvardX/SW12.3x/2016,8.695652
1682,HarvardX/HUM1.6x/3T2015,7.142857
2632,HarvardX/SPU29x/3T2016,5.000000


When k = 4

In [12]:
completionRate4Anon = getAnonCompletionRates(df, quasi_identifier_labels, 4)

Size of 4-anonymous dataset: 12334


In [13]:
completionRate4Anon.head()

,course_id,completionRate
905,HarvardX/HLS4X/3T2017,11.111111
1452,HarvardX/SW12.3x/2016,10.000000
1442,HarvardX/SPU29x/3T2016,6.250000
1446,HarvardX/SPU29x/3T2016,4.166667
1397,HarvardX/PH559x/3T2017,3.773585


When k = 5

In [14]:
completionRate5Anon = getAnonCompletionRates(df, quasi_identifier_labels, 5)

Size of 5-anonymous dataset: 10206


In [15]:
completionRate5Anon.head()

,course_id,completionRate
962,HarvardX/SW12.3x/2016,12.500000
954,HarvardX/SPU29x/3T2016,6.818182
929,HarvardX/PH559x/3T2017,3.773585
928,HarvardX/PH559x/3T2017,3.773585
876,HarvardX/PH556/2015T3,2.040816


### Synthetic Records
- Make the data k-anonymous
- Find the number of synthetic records needed for each case
- Compute completion rates and compare to dataset without synthetic records

When k = 3

In [16]:
def addSyntheticRows(df, quasi_identifier_labels, k):
    synthetic_k_anon_df = df.reset_index()
    not_k_anonymous = synthetic_k_anon_df.groupby(quasi_identifier_labels)\
                      .size().reset_index(name = 'studentCount')
    not_k_anonymous = not_k_anonymous[not_k_anonymous.studentCount < k]

    for i in range(k):
        rowsToAdd = not_k_anonymous[not_k_anonymous.studentCount == i]
        if len(rowsToAdd) > 0:
            for j in range(k - i):
                synthetic_k_anon_df = synthetic_k_anon_df.append(rowsToAdd, ignore_index = True)
            
    rowsAdded = synthetic_k_anon_df.shape[0] - len(df)
    print("Size of synthetic dataset: %d\nSynthetic Rows Added: %d" % (len(synthetic_k_anon_df), rowsAdded))
    # Drop the studentCount column from the dataset we return because df_quasi does not
    # have it.  This column is an artifact of when we grouped synthetic_k_anon_df above
    synthetic_k_anon_df = synthetic_k_anon_df.drop("studentCount", axis = 1)
    return synthetic_k_anon_df
    

Add synthetic data when k = 3

In [17]:
synthetic3AnonDf = addSyntheticRows(df, quasi_identifier_labels, 3)

Size of synthetic dataset: 553624
Synthetic Rows Added: 353625


Add synthetic data when k = 4

In [18]:
synthetic4AnonDf = addSyntheticRows(df_quasi, quasi_identifier_labels, 4)

Size of synthetic dataset: 734054
Synthetic Rows Added: 534055


Add synthetic data when k = 5

In [19]:
synthetic5AnonDf = addSyntheticRows(df_quasi, quasi_identifier_labels, 5)

Size of synthetic dataset: 915016
Synthetic Rows Added: 715017


### K-Anonymity 
- Generalization
    - YoB, nform_* 
- Blurring
    - Last 3 digits of postal_code
- Suprression 
    - Remaining 


In [30]:
df_quasi_copy = df.copy()

In [31]:
def blurring(df, col, lenCol):
    df_quasi_blur = df
    df_quasi_blur.loc[:,col] = df_quasi_blur.loc[:,col].astype(str).str[0:int(lenCol-3)] + "*" * 3;
    return df_quasi_blur

In [32]:
blurring(df_quasi_copy, "postalCode", 5);

In [33]:
def generalization(df, colChange, bucketSize, maxVal):
    col = colChange
    df_quasi_gen = df
    binNum = int(maxVal / bucketSize)
    bins = [bucketSize*i for i in range(NA_FILL_VALUE-1, binNum)]
    df_quasi_gen.loc[:,col] = df_quasi_gen[col].astype(int)
    df_quasi_gen.loc[:,col] = pd.cut(df_quasi_gen[col], bins)
    return df_quasi_gen

In [34]:
r = generalization(df_quasi_copy, "YoB", 10, 2019)
r = generalization(r, "nforum_posts", 10, 470)
r = generalization(r, "nforum_votes", 10, 640)
r = generalization(r, "nforum_endorsed", 10, 50)
r = generalization(r, "nforum_threads", 10, 150)
r = generalization(r, "nforum_comments", 10, 450)
r = generalization(r, "nforum_pinned", 10, 20)

In [35]:
lDiverse = r
lDiverse["grade"] = df["grade"]

In [36]:
lDiverseLevels = lDiverse.groupby(quasi_identifier_labels)["grade"].nunique().reset_index(name = "Grade_Count")
#.reset_index(name = "Grade_Count")
# lDiverseLevels.reset_index(name = "Grade_Count")
lDiverseLevels

,gender,longitude,nforum_threads,un_special_region,LoE,un_economic_group,region,postalCode,registered,nprogcheck,...,nforum_pinned,nforum_posts,nforum_comments,course_id,city,un_major_region,un_developing_nation,nforum_votes,nforum_endorsed,Grade_Count
0,0,0,"(-10, 0]",0,0,0,0,0***,True,0,...,"(-10, 0]","(-10, 0]","(-10, 0]",HarvardX/PH207x/2012_Fall,0,0,0,"(-10, 0]","(-10, 0]",1
1,0,0,"(-10, 0]",0,0,0,0,0***,True,0,...,"(-10, 0]","(-10, 0]","(-10, 0]",HarvardX/AI12.1x/2013_SOND,0,0,0,"(-10, 0]","(-10, 0]",1
2,0,0,"(-10, 0]",0,0,0,0,0***,True,0,...,"(-10, 0]","(-10, 0]","(-10, 0]",HarvardX/CS50x/2014_T1,0,0,0,"(-10, 0]","(-10, 0]",2
3,0,0,"(-10, 0]",0,0,0,0,0***,True,0,...,"(-10, 0]","(-10, 0]","(-10, 0]",HarvardX/ER22.1x/1T2018,0,0,0,"(-10, 0]","(-10, 0]",1
4,0,0,"(-10, 0]",0,0,0,0,0***,True,0,...,"(-10, 0]","(-10, 0]","(-10, 0]",HarvardX/ER22x/2013_Spring,0,0,0,"(-10, 0]","(-10, 0]",1
5,0,0,"(-10, 0]",0,0,0,0,0***,True,0,...,"(-10, 0]","(-10, 0]","(-10, 0]",HarvardX/HDS1544.1x/2013_SOND,0,0,0,"(-10, 0]","(-10, 0]",2
6,0,0,"(-10, 0]",0,0,0,0,0***,True,0,...,"(-10, 0]","(-10, 0]","(-10, 0]",HarvardX/HKS101A/1T2017,0,0,0,"(-10, 0]","(-10, 0]",1
7,0,0,"(-10, 0]",0,0,0,0,0***,True,0,...,"(-10, 0]","(-10, 0]","(-10, 0]",HarvardX/HKS101A/2015T3,0,0,0,"(-10, 0]","(-10, 0]",1
8,0,0,"(-10, 0]",0,0,0,0,0***,True,0,...,"(-10, 0]","(-10, 0]","(-10, 0]",HarvardX/HLS4X/3T2017,0,0,0,"(-10, 0]","(-10, 0]",1
9,0,0,"(-10, 0]",0,0,0,0,0***,True,0,...,"(-10, 0]","(-10, 0]","(-10, 0]",HarvardX/HUM1.1x/3T2015,0,0,0,"(-10, 0]","(-10, 0]",1


In [103]:
lDiverseLevels["Grade_Count"].unique()

array([1, 2, 3, 4, 5, 6])

In [ ]:
df_quasi["completed"] = df["completed"]

In [ ]:
completionRate4Anon_GenBlurSuppress = getAnonCompletionRates(df_quasi, quasi_identifier_labels, 4)

In [ ]:
completionRate5Anon_GenBlurSuppress = getAnonCompletionRates(df_quasi, quasi_identifier_labels, 5) 